In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Dataset
### Data generate

(300, 2)
(300, 3)


In [3]:
N = 100 # number of points per class
D = 2 # dimensionality
K = 3 # number of classes
inputs = np.zeros((N*K,D)) # data matrix (each row = single example)
targets = np.zeros(N*K, dtype='uint8') # class labels
for j in range(K):
    ix = range(N*j,N*(j+1))
    r = np.linspace(0.0,1,N) # radius
    t = np.linspace(j*4,(j+1)*4,N) + np.random.randn(N)*0.2 # theta
    inputs[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
    targets[ix] = j

# # lets visualize the data:
plt.scatter(inputs[:, 0], inputs[:, 1], c=targets, s=40, cmap=plt.cm.Spectral)
plt.show()

one_hot = np.zeros([N*K ,K], dtype=np.int)
for i in range(len(targets)):
    one_hot[i][targets[i]] = 1 
targets = one_hot

print (inputs.shape)
print (targets.shape)

### Shuffling dataset

In [4]:
shuffle_indices = np.arange(len(inputs))
np.random.shuffle(shuffle_indices)

inputs = inputs[shuffle_indices]
targets = targets[shuffle_indices]

### Split the dataset

In [5]:
training_rate = .9

sample_count = len(inputs)
train_sample_count = int(training_rate * sample_count)

train_inputs  = inputs[0:train_sample_count]
train_targets = targets[0:train_sample_count]

test_inputs  = inputs[train_sample_count::]
test_targets = targets[train_sample_count::]

print(train_inputs.shape)
print(train_targets.shape)
print(test_inputs.shape)
print(test_targets.shape)

(270, 2)
(270, 3)
(30, 2)
(30, 3)


# Model

### Architecture :

In [12]:
inputs_units = 2
hidden_units = 100
outputs_units = 3

### Initialization :

In [13]:
W1 = np.random.uniform(-1, 1, size=(inputs_units ,hidden_units))
dW1_s = np.zeros((inputs_units ,hidden_units))

W2 = np.random.uniform(-1, 1, size=(hidden_units ,outputs_units))
dW2_s = np.zeros((hidden_units ,outputs_units))

b1 = np.random.uniform(-1, 1, size=(1, hidden_units))
db1_s = np.zeros((1 ,hidden_units))

b2 = np.random.uniform(-1, 1, size=(1, outputs_units))
db2_s = np.zeros((1, outputs_units))

### Train Model :

In [8]:
epochs = 100
batch_size = 5
learning_rate = 0.01
beta = 0.9
epsilon = 1e-8

num_examples = train_inputs.shape[0]

for epoch in range(epochs):
    iter_loss = 0.
    iter_accuracy = 0.
    iteration = 0
    for batch_index in range(0, num_examples, batch_size):
        iteration += 1
        if (batch_index + batch_size) > num_examples:
            X = train_inputs[batch_index::]
            y = train_targets[batch_index::]
        else:
            X = train_inputs[batch_index:batch_index + batch_size]
            y = train_targets[batch_index:batch_index + batch_size]
        
        hidden_layer = np.maximum(0, np.dot(X, W1) + b1)
        scores = np.dot(hidden_layer, W2) + b2

        normalize_scores = scores - scores.max()
        exp_scores = np.exp(normalize_scores)
        y_hat = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

        loss = -np.sum(y * np.log(y_hat)) / batch_size
        iter_loss += loss

        d_loss = y_hat - y
        d_loss /= batch_size 

        dW2 = np.dot(hidden_layer.T, d_loss)
        dW2_s = beta * dW2_s + (1-beta) * dW2 ** 2
        db2 = np.sum(d_loss, axis=0, keepdims=True)
        db2_s = beta * db2_s + (1-beta) * db2 ** 2

        dhidden = np.dot(d_loss, W2.T)
        dhidden[hidden_layer <= 0] = 0
        
        dW1 = np.dot(X.T, dhidden)
        dW1_s = beta * dW1_s + (1-beta) * dW1 ** 2
        db1 = np.sum(dhidden, axis=0, keepdims=True)
        db1_s = beta * db1_s + (1-beta) * db1 ** 2

       
        W1 += -(learning_rate / np.sqrt(dW1_s + epsilon)) * dW1
        b1 += -(learning_rate / np.sqrt(db1_s + epsilon)) * db1
        W2 += -(learning_rate / np.sqrt(dW2_s + epsilon)) * dW2
        b2 += -(learning_rate / np.sqrt(db2_s + epsilon)) * db2

        accuracy = (np.argmax(y, axis=1) == np.argmax(y_hat, axis=1)).mean()
        iter_accuracy += accuracy
        
    iter_loss /= iteration
    iter_accuracy /= iteration
    
    if epoch % 10 ==0:
        print('Epoch [{}], Loss: {:.4f}, Accuracy: {:.1f}%'.format(epoch, iter_loss, 100*iter_accuracy))

Epoch [0], Loss: 0.8207, Accuracy: 60.0%
Epoch [10], Loss: 0.1344, Accuracy: 96.3%
Epoch [20], Loss: 0.0675, Accuracy: 97.8%
Epoch [30], Loss: 0.0436, Accuracy: 98.5%
Epoch [40], Loss: 0.0335, Accuracy: 98.1%
Epoch [50], Loss: 0.0285, Accuracy: 98.9%
Epoch [60], Loss: 0.0268, Accuracy: 98.9%
Epoch [70], Loss: 0.0275, Accuracy: 98.9%
Epoch [80], Loss: 0.0267, Accuracy: 98.9%
Epoch [90], Loss: 0.0262, Accuracy: 98.9%


### Test Model :

In [11]:
X = test_inputs
y = test_targets

num_examples = len(y)

hidden_layer = np.maximum(0, np.dot(X, W1) + b1)
scores = np.dot(hidden_layer, W2) + b2

normalize_scores = scores - scores.max()
exp_scores = np.exp(normalize_scores)
y_hat = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

loss = -np.sum(y * np.log(y_hat)) / num_examples
accuracy = (np.argmax(y, axis=1) == np.argmax(y_hat, axis=1)).mean()

print('Loss: {:.4f}, Accuracy: {:.1f}%'.format(loss, 100*accuracy))

Loss: 0.0002, Accuracy: 100.0%
